In [1]:
%%time

import numpy as np
from scipy import sparse

CPU times: user 52.4 ms, sys: 25.7 ms, total: 78.1 ms
Wall time: 176 ms


In [2]:
%%time

# jester file should look like
'''
7302,29,7.156
61815,46,6.375
31128,96,2.281
36125,147,-1.781
18007,60,2.188
7387,99,3.594
12007,18,-2.094
'''

m = 63_978
n = 150
nnz_train = 1_000_000
nnz_test = 761_439

train_data_file = "jester_train.csv"
test_data_file = "dont_use.csv"

print("preparing test data")
test_user, test_item, test_rating = np.loadtxt(test_data_file, delimiter=',',
                                               dtype=[('f0', np.int32), ('f1', np.int32), ('f2', np.float)],
                                               skiprows=1, unpack=True)

print("preparing training data")
train_user, train_item, train_rating = np.loadtxt(train_data_file, delimiter=',',
                                                  dtype=[('f0', np.int32), ('f1', np.int32), ('f2', np.float)],
                                                  skiprows=1, unpack=True)

preparing test data
preparing training data
CPU times: user 7.57 s, sys: 114 ms, total: 7.69 s
Wall time: 7.67 s


In [3]:
%%time

print(test_user)
print(test_item)
print(test_rating)
print("")
print(np.max(test_user))
print(np.max(test_item))
print(np.max(test_rating))
print("")
print(np.min(test_user))
print(np.min(test_item))
print(np.min(test_rating))
print("")
print(np.unique(test_user).size)
print(np.unique(test_item).size)
print(np.unique(test_rating).size)
print("")
print(test_user.size)

assert test_user.size == nnz_test

[30762 54667 38515 ... 63826 41898  1862]
[ 24 128  68 ... 108  42  78]
[-0.188  9.844  8.5   ...  2.031 -1.25  -7.062]

63978
150
10.0

1
5
-10.0

57249
140
641

761439
CPU times: user 120 ms, sys: 16 ms, total: 136 ms
Wall time: 130 ms


In [4]:
%%time

print(train_user)
print(train_item)
print(train_rating)
print("")
print(np.max(train_user))
print(np.max(train_item))
print(np.max(train_rating))
print("")
print(np.min(train_user))
print(np.min(train_item))
print(np.min(train_rating))
print("")
print(np.unique(train_user).size)
print(np.unique(train_item).size)
print(np.unique(train_rating).size)
print("")
print(train_user.size)

assert train_user.size == nnz_train
assert np.max(train_user) == m
assert np.max(train_item) == n

[ 7302 61815 31128 ... 40650 45014 22566]
[29 46 96 ... 98 19  7]
[ 7.156  6.375  2.281 ...  6.188 -4.906  1.781]

63978
150
10.0

1
5
-10.0

58125
140
641

1000000
CPU times: user 166 ms, sys: 8.05 ms, total: 174 ms
Wall time: 167 ms


In [5]:
%%time

#1-based to 0-based
R_test_coo = sparse.coo_matrix((test_rating, (test_user - 1, test_item - 1)))
assert R_test_coo.nnz == nnz_test

outfile_test = open("test.txt", 'w')
for i in range(nnz_test):
    outfile_test.write(str(test_user[i]) + " " + str(test_item[i]) + " " + str(test_rating[i]) + "\n")

CPU times: user 1.72 s, sys: 23.8 ms, total: 1.74 s
Wall time: 1.72 s


In [6]:
%%time

# for test data, we need COO format to calculate test RMSE

R_test_coo.data.astype(np.float32).tofile('R_test_coo.data.bin')
R_test_coo.row.tofile('R_test_coo.row.bin')
R_test_coo.col.tofile('R_test_coo.col.bin')

test_data = np.fromfile('R_test_coo.data.bin', dtype=np.float32)
test_row = np.fromfile('R_test_coo.row.bin', dtype=np.int32)
test_col = np.fromfile('R_test_coo.col.bin', dtype=np.int32)

CPU times: user 2.93 ms, sys: 12.1 ms, total: 15 ms
Wall time: 17.2 ms


In [7]:
print(R_test_coo.data)
print(R_test_coo.row)
print(R_test_coo.col)
print("")
print(test_data)
print(test_row)
print(test_col)

[-0.188  9.844  8.5   ...  2.031 -1.25  -7.062]
[30761 54666 38514 ... 63825 41897  1861]
[ 23 127  67 ... 107  41  77]

[-0.188  9.844  8.5   ...  2.031 -1.25  -7.062]
[30761 54666 38514 ... 63825 41897  1861]
[ 23 127  67 ... 107  41  77]


In [8]:
%%time

print(np.max(R_test_coo.data))
print(np.max(R_test_coo.row))
print(np.max(R_test_coo.col))
print("")
print(np.min(R_test_coo.data))
print(np.min(R_test_coo.row))
print(np.min(R_test_coo.col))
print("")
print(np.unique(test_user).size)
print(np.unique(R_test_coo.row).size)
print(np.unique(test_item).size)
print(np.unique(R_test_coo.col).size)

10.0
63977
149

-10.0
0
4

57249
57249
140
140
CPU times: user 2.71 s, sys: 7.69 ms, total: 2.72 s
Wall time: 302 ms


In [9]:
%%time

#1-based to 0-based
R_train_coo = sparse.coo_matrix((train_rating, (train_user - 1, train_item - 1)))
assert R_train_coo.nnz == nnz_train

outfile_train = open("train.txt", 'w')
for i in range(nnz_train):
    outfile_train.write(str(train_user[i]) + " " + str(train_item[i]) + " " + str(train_rating[i]) + "\n")

CPU times: user 4.68 s, sys: 32.3 ms, total: 4.71 s
Wall time: 2.35 s


In [10]:
%%time

# for training data, we need COO format to calculate training RMSE
# we need CSR format R when calculate X from \Theta
# we need CSC format of R when calculating \Theta from X
R_train_coo.data.astype(np.float32).tofile('R_train_coo.data.bin')
R_train_coo.row.tofile('R_train_coo.row.bin')
R_train_coo.col.tofile('R_train_coo.col.bin')

R_train_csr = R_train_coo.tocsr()
R_train_csc = R_train_coo.tocsc()

R_train_csr.data.astype(np.float32).tofile('R_train_csr.data.bin')
R_train_csr.indices.tofile('R_train_csr.indices.bin')
R_train_csr.indptr.tofile('R_train_csr.indptr.bin')
R_train_csc.data.astype(np.float32).tofile('R_train_csc.data.bin')
R_train_csc.indices.tofile('R_train_csc.indices.bin')
R_train_csc.indptr.tofile('R_train_csc.indptr.bin')

CPU times: user 175 ms, sys: 28.1 ms, total: 203 ms
Wall time: 205 ms


In [11]:
%%time

train_data = np.fromfile('R_train_coo.data.bin', dtype=np.float32)
train_row = np.fromfile('R_train_coo.row.bin', dtype=np.int32)
train_col = np.fromfile('R_train_coo.col.bin', dtype=np.int32)

train_csc_data = np.fromfile('R_train_csc.data.bin', dtype=np.float32)
train_csc_indices = np.fromfile('R_train_csc.indices.bin', dtype=np.int32)
train_csc_indptr = np.fromfile('R_train_csc.indptr.bin', dtype=np.int32)

train_csr_data = np.fromfile('R_train_csr.data.bin', dtype=np.float32)
train_csr_indices = np.fromfile('R_train_csr.indices.bin', dtype=np.int32)
train_csr_indptr = np.fromfile('R_train_csr.indptr.bin', dtype=np.int32)

CPU times: user 0 ns, sys: 15.9 ms, total: 15.9 ms
Wall time: 14.7 ms


In [12]:
print(R_train_coo.data)
print(R_train_coo.row)
print(R_train_coo.col)
print("")
print(train_data)
print(train_row)
print(train_col)
print("")
print(R_train_csr.data)
print(R_train_csr.indices)
print(R_train_csr.indptr)
print("")
print(train_csr_data)
print(train_csr_indices)
print(train_csr_indptr)
print("")
print(R_train_csc.data)
print(R_train_csc.indices)
print(R_train_csc.indptr)
print("")
print(train_csc_data)
print(train_csc_indices)
print(train_csc_indptr)

[ 7.156  6.375  2.281 ...  6.188 -4.906  1.781]
[ 7301 61814 31127 ... 40649 45013 22565]
[28 45 95 ... 97 18  6]

[ 7.156  6.375  2.281 ...  6.188 -4.906  1.781]
[ 7301 61814 31127 ... 40649 45013 22565]
[28 45 95 ... 97 18  6]

[-9.281 -9.281  0.875 ...  8.5    8.375  8.938]
[  6   7  14 ... 143 144 145]
[      0      32      48 ...  999922  999927 1000000]

[-9.281 -9.281  0.875 ...  8.5    8.375  8.938]
[  6   7  14 ... 143 144 145]
[      0      32      48 ...  999922  999927 1000000]

[-9.844 -5.812 -0.031 ...  1.531  2.469 -1.188]
[    2     3     5 ... 63949 63954 63959]
[      0       0       0       0       0     373     373   33963   66622
   66622   66622   66622   66622   98617   98617  129758  160613  190717
  220193  248948  249260  255797  259931  263747  266331  271156  276508
  276708  282253  290403  293467  293606  306496  308995  314178  325984
  335118  338663  343362  348364  352418  356222  361380  361477  364268
  368271  373764  382291  388588  396823  406283 

In [13]:
%%time

print(np.max(R_train_coo.data))
print(np.max(R_train_coo.row))
print(np.max(R_train_coo.col))
print("")
print(np.min(R_train_coo.data))
print(np.min(R_train_coo.row))
print(np.min(R_train_coo.col))
print("")
print(np.unique(train_user).size)
print(np.unique(R_train_coo.row).size)
print(np.unique(train_item).size)
print(np.unique(R_train_coo.col).size)

10.0
63977
149

-10.0
0
4

58125
58125
140
140
CPU times: user 2.8 s, sys: 3.48 ms, total: 2.8 s
Wall time: 273 ms


In [14]:
%%time

print("write extra meta file")

outfile_meta = open("meta_modified_all", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n" + str(nnz_train) + "\n")
outfile_meta.write("""R_train_coo.data.bin
R_train_coo.row.bin
R_train_coo.col.bin
R_train_csr.indptr.bin
R_train_csr.indices.bin
R_train_csr.data.bin
R_train_csc.indptr.bin
R_train_csc.indices.bin
R_train_csc.data.bin
""")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

write extra meta file
CPU times: user 1.67 ms, sys: 8 µs, total: 1.67 ms
Wall time: 1.07 ms


In [15]:
%%time

print("writing extra meta file")

outfile_meta = open("meta", 'w')
outfile_meta.write(str(m) + " " + str(n) + "\n")
outfile_meta.write(str(nnz_train) + " " + "train.txt\n")
outfile_meta.write(str(nnz_test) + " " + "test.txt\n")

writing extra meta file
CPU times: user 2.35 ms, sys: 41 µs, total: 2.39 ms
Wall time: 558 µs
